In [9]:
import os
import pandas as pd

In [6]:
os.chdir("../")

In [7]:
os.getcwd()

'/Users/tunahankilic/Desktop/NBAGamePrediction'

In [216]:
df = pd.read_parquet("data/nba_games.parquet")

In [217]:
df = df.loc[df['season'] > '2018'].sort_values("date").reset_index(drop=True)

In [218]:
del df["index_opp"]

In [219]:
df = df.dropna(axis=1, how='all')

In [155]:
df.columns

Index(['mp', 'fg', 'fga', 'fg%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%', 'orb',
       'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'ts%', 'efg%',
       '3par', 'ftr', 'orb%', 'drb%', 'trb%', 'ast%', 'stl%', 'blk%', 'tov%',
       'usg%', 'ortg', 'drtg', 'pace', 'ft/fga', 'team', 'total', 'home',
       'mp_opp', 'fg_opp', 'fga_opp', 'fg%_opp', '3p_opp', '3pa_opp',
       '3p%_opp', 'ft_opp', 'fta_opp', 'ft%_opp', 'orb_opp', 'drb_opp',
       'trb_opp', 'ast_opp', 'stl_opp', 'blk_opp', 'tov_opp', 'pf_opp',
       'pts_opp', 'ts%_opp', 'efg%_opp', '3par_opp', 'ftr_opp', 'orb%_opp',
       'drb%_opp', 'trb%_opp', 'ast%_opp', 'stl%_opp', 'blk%_opp', 'tov%_opp',
       'usg%_opp', 'ortg_opp', 'drtg_opp', 'pace_opp', 'ft/fga_opp',
       'team_opp', 'total_opp', 'home_opp', 'season', 'date', 'won'],
      dtype='object')

In [220]:
def create_target(team: pd.DataFrame):
    team["target"] = team["won"].shift(-1)
    team["target"] = team["target"].map({True: 1, False: 0}).fillna(-1).astype(int)
    return team


df = df.groupby("team", group_keys=False).apply(create_target)

In [221]:
def calculate_days_difference(team: pd.DataFrame):
    team['days_to_next_game'] = team['date'].diff().dt.days.shift(-1)
    return team


df = df.groupby(['team', 'season'], group_keys=False).apply(calculate_days_difference)

In [222]:
removed_columns = ["season", "date", "won", "target", "team", "team_opp"]
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [159]:
selected_columns

Index(['mp', 'fg', 'fga', 'fg%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%', 'orb',
       'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'ts%', 'efg%',
       '3par', 'ftr', 'orb%', 'drb%', 'trb%', 'ast%', 'stl%', 'blk%', 'tov%',
       'usg%', 'ortg', 'drtg', 'pace', 'ft/fga', 'total', 'home', 'mp_opp',
       'fg_opp', 'fga_opp', 'fg%_opp', '3p_opp', '3pa_opp', '3p%_opp',
       'ft_opp', 'fta_opp', 'ft%_opp', 'orb_opp', 'drb_opp', 'trb_opp',
       'ast_opp', 'stl_opp', 'blk_opp', 'tov_opp', 'pf_opp', 'pts_opp',
       'ts%_opp', 'efg%_opp', '3par_opp', 'ftr_opp', 'orb%_opp', 'drb%_opp',
       'trb%_opp', 'ast%_opp', 'stl%_opp', 'blk%_opp', 'tov%_opp', 'usg%_opp',
       'ortg_opp', 'drtg_opp', 'pace_opp', 'ft/fga_opp', 'total_opp',
       'home_opp', 'days_to_next_game'],
      dtype='object')

In [223]:
def rolling_averages(team: pd.DataFrame, window: int):
    rolling = team.rolling(window=window).mean()
    return rolling


df_rolling = df.groupby(["team", "season"], group_keys=False)[selected_columns].apply(rolling_averages, window=10)

In [224]:
rolling_cols = [f'{col}_10' for col in df_rolling.columns]
df_rolling.columns = rolling_cols

df = pd.concat([df, df_rolling], axis=1)

In [225]:
df_filtered = df.drop(df[df['target'] == -1].index).reset_index(drop=True).dropna().reset_index(drop=True)

In [226]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    df = df.copy()
    return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))


df_filtered["home_next"] = add_col(df_filtered, "home")
df_filtered["team_opp_next"] = add_col(df_filtered, "team_opp")
df_filtered["date_next"] = add_col(df_filtered, "date")

In [227]:
df_filtered[['team', 'team_opp', 'date', 'team_opp_next', 'date_next']]

,team,team_opp,date,team_opp_next,date_next
0,PHI,LAC,2019-01-01,PHO,2019-01-02
1,CHO,DAL,2019-01-02,DEN,2019-01-05
2,DAL,CHO,2019-01-02,BOS,2019-01-04
3,LAL,OKC,2019-01-02,NYK,2019-01-04
4,PHI,PHO,2019-01-02,DAL,2019-01-05
...,...,...,...,...,...
10169,DEN,MIA,2023-06-04,MIA,2023-06-07
10170,MIA,DEN,2023-06-07,DEN,2023-06-09
10171,DEN,MIA,2023-06-07,MIA,2023-06-09
10172,DEN,MIA,2023-06-09,None,NaT


In [228]:
df_full = df_filtered.merge(
    df_filtered[rolling_cols + ["team_opp_next", "date_next", "team"]],
    left_on = ["team", "date_next"],
    right_on = ["team_opp_next", "date_next"],
    suffixes=('', '_nopp')
)

In [229]:
df_full[['team', 'team_opp', 'date', 'team_opp_next', 'date_next', 'days_to_next_game_10', 'team_nopp', 'team_opp_next_nopp', 'days_to_next_game_10_nopp']]

,team,team_opp,date,team_opp_next,date_next,days_to_next_game_10,team_nopp,team_opp_next_nopp,days_to_next_game_10_nopp
0,PHI,PHO,2019-01-02,DAL,2019-01-05,7.9,DAL,PHI,7.7
1,SAC,DEN,2019-01-03,GSW,2019-01-05,8.0,GSW,SAC,8.1
2,GSW,HOU,2019-01-03,SAC,2019-01-05,8.1,SAC,GSW,8.0
3,NYK,LAL,2019-01-04,POR,2019-01-07,8.2,POR,NYK,7.9
4,LAL,NYK,2019-01-04,MIN,2019-01-06,7.8,MIN,LAL,8.1
...,...,...,...,...,...,...,...,...,...
9949,MIA,DEN,2023-06-01,DEN,2023-06-04,2.5,DEN,MIA,3.4
9950,MIA,DEN,2023-06-04,DEN,2023-06-07,2.6,DEN,MIA,3.3
9951,DEN,MIA,2023-06-04,MIA,2023-06-07,3.3,MIA,DEN,2.6
9952,MIA,DEN,2023-06-07,DEN,2023-06-09,2.3,DEN,MIA,3.3


In [230]:
df_full.columns[df_full.dtypes == "object"]

Index(['team', 'team_opp', 'season', 'team_opp_next', 'team_opp_next_nopp',
       'team_nopp'],
      dtype='object')

In [412]:
train = df_full.loc[df_full["season"] < "2023"]
test = df_full.loc[df_full["season"] == "2023"]

In [458]:
df_full.loc[df_full["season"] < "2023"].to_parquet('data/train.parquet')
df_full.loc[df_full["season"] == "2023"].to_parquet('data/test.parquet')

In [232]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier

In [378]:
rc_classifier = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)

sfs = SequentialFeatureSelector(rc_classifier, n_features_to_select=30, direction="forward", cv=split)

In [379]:
removed_cols = ['team', 'team_opp', 'season', 'date', 'won', 'team_opp_next',
       'date_next', 'team_opp_nopp', 'team_nopp', "team_opp_next_nopp", "target"]

selected_cols = df_full.columns[~df_full.columns.isin(removed_cols)]

In [455]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [413]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train.loc[:, selected_cols] = scaler.fit_transform(train[selected_cols])
test.loc[:, selected_cols] = scaler.transform(test[selected_cols])

In [456]:
def backtest(train, test, model, predictors, season=2023):
    all_predictions = []
    model.fit(train[predictors], train['target'])
    #train_preds = model.predict(train[predictors])
    print(f'Season - {season}')
    #print(f'Train Accuracy - {accuracy_score(train["target"], train_preds)}')
    preds = model.predict(test[predictors])
    print(f'Test Accuracy - {accuracy_score(test["target"], preds)}')
    print(f'Precision: {precision_score(test["target"], preds)}')
    print(f'Recall: {recall_score(test["target"], preds)}')
    preds = pd.Series(preds, index=test.index)
    combined = pd.concat([test["target"], preds], axis=1)
    combined.columns = ['actual', 'prediction']
    all_predictions.append(combined)
    return pd.concat(all_predictions)

In [382]:
sfs.fit(train[selected_cols], train["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [383]:
cols = list(selected_cols[sfs.get_support()])

In [384]:
cols

['drb',
 'pf',
 'usg%',
 'fg_opp',
 '3p_opp',
 'tov_opp',
 'usg%_opp',
 'days_to_next_game',
 'tov%_10',
 'usg%_10',
 'ortg_10',
 'drtg_10',
 'pace_10',
 'fg_opp_10',
 '3p%_opp_10',
 'ft%_opp_10',
 'pts_opp_10',
 'usg%_opp_10',
 'ortg_opp_10',
 'drtg_opp_10',
 'pace_opp_10',
 'total_opp_10',
 'home_next',
 'fg_10_nopp',
 '3p%_10_nopp',
 'trb_10_nopp',
 'usg%_10_nopp',
 'ortg_10_nopp',
 'usg%_opp_10_nopp',
 'drtg_opp_10_nopp']

In [421]:
import optuna
from optuna.samplers import TPESampler

def run_optimization(data: pd.DataFrame, columns: list, num_trials: int):

    train = data[data["season"] < '2022']
    val = data[data["season"] == "2022"]

    def objective(trial):
        params = {
            'n_estimators': 1000,
            'max_depth': trial.suggest_int('max_depth', 1, 20, 1),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 10, 1),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 4, 1),
            'random_state': 2023,
            'n_jobs': -1
        }

        rf = RandomForestClassifier(**params)
        rf.fit(train[cols], train["target"])
        y_pred = rf.predict(val[cols])
        accuracy = accuracy_score(val["target"], y_pred)

        return accuracy

    sampler = TPESampler(seed=2023)
    study = optuna.create_study(direction="maximize", sampler=sampler)
    study.optimize(objective, n_trials=num_trials)

In [431]:
from sklearn.model_selection import cross_val_score
import numpy as np

In [459]:
def run_optimization(data: pd.DataFrame, columns: list, num_trials: int):

    def objective(trial):

        X = train[columns]
        y = train["target"]
        # Define hyperparameter search space
        n_estimators = 1000
        max_depth = trial.suggest_int('max_depth', 3, 15)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10, 1)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 5, 1)
        
        # Create the model with suggested hyperparameters
        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=2023
        )
        
        # Define TimeSeriesSplit cross-validation
        tscv = TimeSeriesSplit(n_splits=5)
        
        # Calculate the performance metric using cross_val_score and TimeSeriesSplit
        scores = cross_val_score(model, X, y, cv=tscv, scoring='accuracy')
        
        # Return the average accuracy (Optuna maximizes the objective)
        return np.mean(scores)
    
    study = optuna.create_study(direction='maximize')  # We want to maximize the accuracy
    study.optimize(objective, n_trials=num_trials)  # You can increase the number of trials for a more thorough search
    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial
    print("  Value: ", trial.value)
    print("  Params: ")
    for key, value in trial.params.items():
            print("    {}: {}".format(key, value))
    return trial

In [461]:
best = run_optimization(train, cols, 1)

[I 2023-07-30 14:38:04,944] A new study created in memory with name: no-name-68bf36d7-b653-44b6-a6be-bb935b4738b4
[I 2023-07-30 14:38:19,726] Trial 0 finished with value: 0.6034428794992175 and parameters: {'max_depth': 5, 'min_samples_split': 10, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.6034428794992175.


Number of finished trials:  1
Best trial:
  Value:  0.6034428794992175
  Params: 
    max_depth: 5
    min_samples_split: 10
    min_samples_leaf: 3


In [463]:
best.params

{'max_depth': 5, 'min_samples_split': 10, 'min_samples_leaf': 3}

In [423]:
from sklearn.ensemble import RandomForestClassifier

In [451]:
rf = RandomForestClassifier(max_depth=5, min_samples_split=6, min_samples_leaf=1, n_estimators=1000, random_state=2023)

In [457]:
predictions = backtest(train, test, rf, cols)

Season - 2023
Test Accuracy - 0.5853765323992994
Precision: 0.6145710928319624
Recall: 0.45796847635726795
